In [ ]:
### Librerias ###
# manejo de datos
import pandas as pd
import numpy as np

# Data Science
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,RandomizedSearchCV

# Estadistica
from scipy.stats import uniform

# Graficos
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Defino indices para identificar las trx
df.set_index(['identificador_trx_sospechosa1', 'identificador_trx_sospechosa2'], inplace=True)


In [ ]:
# Diviod los datos en train y test
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
### ENTRENO MODELO ###

# Estandarizacion
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# defino el modelo base
model = IsolationForest(random_state=10)

# Hiperparametros a optimizar
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'contamination': uniform(0.01, 0.1),  # Proporción esperada de anomalías
    'max_samples': ['auto', 0.5, 0.75, 1.0],
    'max_features': [0.5, 0.75, 1.0]
}

# funcion de evaluación personalizada para Isolation Forest
def scoring_function(model, X, y=None):
    scores = model.decision_function(X)  # Puntajes de anomalía
    return np.mean(scores)  # Se busca el modelo con mayor puntaje promedio

'''Isolation Forest
 seleccionará automáticamente los mejores hiperparámetros
 basándose en la media de los puntajes de anomalía'''

# Optimizacion por Randomized Search
random_search = RandomizedSearchCV(
    model, param_distributions=param_dist, n_iter=20, 
    scoring=scoring_function, cv=3, random_state=10
)

random_search.fit(X_train_scaled)  # Entrenar con datos escalados

# Muestros los mejores parametros
print("Mejores hiperparámetros encontrados:")
print(random_search.best_params_)



# Predecir anomalias en datos de test
X_test["anomaly_score"] = best_model.decision_function(X_test_scaled)
X_test["fraude"] = best_model.predict(X_test_scaled)  # -1 = Fraude, 1 = Normal

# Muestros las anomalias en los datos de test
anomalies = X_test[X_test["fraude"] == -1]
print(f"Total de anomalías detectadas: {len(anomalies)}")



In [ ]:
# Me quedo con el mejor modelo
best_model = random_search.best_estimator_


In [ ]:
# Predecir anomalias en datos de test
X_test["anomaly_score"] = best_model.decision_function(X_test_scaled)
X_test["fraude"] = best_model.predict(X_test_scaled)  # -1 = Fraude, 1 = Normal

# Muestros las anomalias en los datos de test
anomalies = X_test[X_test["fraude"] == -1]
print(f"Total de anomalías detectadas: {len(anomalies)}")

In [ ]:
# Crear figura
plt.figure(figsize=(14, 7))

# Graficar transacciones normales
sns.scatterplot(
    data=X_test[X_test["fraude"] == 1],  # Transacciones normales
    x="FechaTrx", y="ImporteOperacion",
    label="Normal", color="blue", alpha=0.7, edgecolor='none', s=100
)

# Graficar anomalías detectadas
sns.scatterplot(
    data=anomalies,  # Solo anomalías
    x="FechaTrx", y="ImporteOperacion",
    label="Anomalía", color="red", alpha=0.8, edgecolor="black", s=50
)

# Configuración del gráfico
plt.title("Detección de Anomalías por Fecha", fontsize=16, fontweight="bold")
plt.xlabel("FechaTrx")
plt.ylabel("ImporteOperacion")

# Cambiar formato del eje Y para mejor lectura
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:,.0f}'))

plt.legend(title="Tipo Trx", loc="upper left")
plt.xticks(rotation=45)  # Rotar las fechas para mejor visibilidad
plt.grid(True, linestyle="--", alpha=0.5)

plt.show()


